In [518]:
type Variable
    name::AbstractString
    domain::Array
    evidence_index::Int32
    assignment_index::Int32
    
    add_values::Function
    value_index::Function
    domain_size::Function
    set_assignment::Function
    get_assignment_index::Function
    get_evidence::Function
    
    function Variable(name="", domain=[])
        this = new()
        
        this.name = name
        this.domain = copy(domain)
        
        this.evidence_index = 0
        this.assignment_index = 0
        
#         Проверено
        this.add_values = function(values::Array)
            this.domain = vcat(this.domain, value)
        end
        
#       Проверено
        this.value_index = function(value::Any)
            return findfirst(this.domain, value)
        end
        
#         Проверено
        this.domain_size =  function()
            return size(this.domain)[1]
        end
        
#         Проверено
        this.set_assignment = function(value)
            this.assignment_index = this.value_index(value)
        end
        
#         Проверно
        this.get_assignment_index = function()
           return this.assignment_index 
        end
        
        
        this.get_evidence = function()
            return this.domain[this.evidence_index]
        end
        return this
    end
    
end

In [517]:
workspace()

In [519]:
type Factor
    name::AbstractString
    scope::Array
    size::Int64
    values::Array
    
    get_value::Function
    get_scope::Function
    add_scope::Function
    add_values::Function
    add_value_at_current_assignment::Function
    get_value_at_current_assignment::Function
    
    function Factor(name="NEW", scope=[])
        this = new()
        
        this.name = name
        this.scope = scope
        
        size = 1
        for v in scope
            size *= v.domain_size()
        end
        this.values = zeros(size)
        
        this.size = size
        
#         Проверено
        this.get_scope = function ()
            return this.scope
        end
        
        
        this.add_scope = function (added::Array)
            this.scope = vcat(this.scope, added)
        end
        
#         this.add = function (value::Node)
#             this.value = value
#         end
        
        
#        fixed bag with indexes
        this.add_values =  function(values::Array)
            for val in values
               idx = 0
                for v in this.scope
                    idx = idx * v.domain_size() + v.value_index(val[1]) -1 
                    print(idx)
                    val = val[2:end]
                end
                idx += 1
                this.values[idx] = val[1]
            end
        end
        
        this.get_value = function(variable_values)
           idx = 0
            print(this.scope)
            for v in this.scope
                idx = idx * v.domain_size() + v.value_index(variable_values[1]) - 1
                variable_values = variable_values[2:end]
            end
            idx += 1
            return this.values[idx]
        end
        
#       As I checked - it is work
        this.add_value_at_current_assignment = function(number)
            idx = 0
            
            for v in this.scope
               idx = idx * v.domain_size() + v.get_assignment_index() -1 
            end
            idx += 1
            this.values[idx] =  number
        end
        
        
        this.get_value_at_current_assignment = function()
            
            idx = 0
            
            for v in this.scope
               idx = idx * v.domain_size() + v.get_assignment_index() - 1
            end
            idx += 1
            return this.values[idx]
        end
        
        
        return this
    end
end

In [520]:
function restrict_factor(F, var, value)
    newFactorName = F.name * "_{R:}" * var.name * " = " * string(value) * "}"
    newFactorScope = copy(F.get_scope())
    
    var.set_assignment(value)
    deleteat!(newFactorScope, findin(newFactorScope, [var]))
    
    newFactor = Factor(newFactorName, newFactorScope)
    
    rec_restrict_factor(F, newFactorScope, newFactor)
    
    return newFactor
end

restrict_factor (generic function with 1 method)

In [521]:
function rec_restrict_factor(oldFactor, newFactorScope, newFactor)
    if isempty(newFactorScope)
        old_assign = oldFactor.get_value_at_current_assignment()
        newFactor.add_value_at_current_assignment(old_assign)
    else
        print(newFactorScope)
        for var_val in newFactorScope[1].domain
            newFactorScope[1].set_assignment(var_val)
            rec_restrict_factor(oldFactor, newFactorScope[2:end], newFactor)
        end
    end
end

rec_restrict_factor (generic function with 1 method)

In [524]:
function step1(NetFactors, EvidenceVars)
    for i in range(1, size(NetFactors)[1])
        for e_v in EvidenceVars
            if e_v in NetFactors[i].get_scope()
                newFactor = restrict_factor(NetFactors[i], e_v, e_v.get_evidence())
                
                NetFactors[i] = copy(newFactor)
            end
        end
    end
end

step1 (generic function with 1 method)

In [495]:
V = Variable("V", [0,1 ])

Variable("V",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [496]:
G = Variable("G", [0,1])
R = Variable("R", [0,1])
S = Variable("S", [0,1])

Variable("S",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [497]:
fac_V = Factor("V", [V])
fac_G = Factor("G", [G])
fac_R = Factor("R", [V, G, R])
fac_S = Factor("S", [G, S])

Factor("S",[Variable("G",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("S",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],4,[0.0,0.0,0.0,0.0],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [498]:
fac_V.add_values([(0, 0.33), (1, 0.66)])
fac_G.add_values([(0, 0.25), (1, 0.75)])
fac_R.add_values([(0, 0, 0, 0.6), (0, 0, 1, 0.4),
        (0, 1, 0, 0.3), (0, 1, 1, 0.7),
        (1,  0, 0, 0.2), (1, 0, 1, 0.8),
        (1, 1, 0, 0.5), (1, 1, 1, 0.5)])
fac_S.add_values([(0, 0, 0.2), (0, 1, 0.8),
        (1, 0, 0.5), (1, 1, 0.5)])

010100000101201312412513613700011213

In [499]:
fac_R

Factor("R",[Variable("V",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("G",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],8,[0.6,0.4,0.3,0.7,0.2,0.8,0.5,0.5],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [500]:
V.assignment_index

0

In [509]:
restrict_factor(fac_R, V, 1).get_value([0, 0])

[Variable("G",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))][Variable("R",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))][Variable("R",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))][Variable("G",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))]

0.2

In [506]:
newF = restrict_factor(fac_R, V, 1)

[Variable("G",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))][Variable("R",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))][Variable("R",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))]

Factor("R_{R:}V = 1}",[Variable("G",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],4,[0.2,0.8,0.5,0.5],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [507]:
fac_R.get_value_at_current_assignment()

0.5

In [503]:
fac_R.get_value([0, 1, 1])

[Variable("V",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("G",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],0,0,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))]

0.7

In [510]:
fac_R

Factor("R",[Variable("V",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("G",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function)),Variable("R",[0,1],0,2,(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))],8,[0.6,0.4,0.3,0.7,0.2,0.8,0.5,0.5],(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function),(anonymous function))

In [516]:
for i in range(1,size([1,2,32,4,5,4,5,45,4])[1])
   println(i) 
end

1
2
3
4
5
6
7
8
9
